In [1]:
import pandas as pd
import os

In [2]:
s3_prefix = "s3://aegovan-data/pubmed_asbtract/predictions_multi_00/"
s3_data ="s3://aegovan-data/human_output/human_interactions_ppi_v2.json"

In [3]:
local_temp = "temp"
local_temp_pred_dir = os.path.join( local_temp, "pred_results")
local_temp_wk_dir = os.path.join( local_temp, "wk")

In [4]:
!rm -rf $local_temp
!mkdir -p $local_temp_pred_dir
!mkdir -p $local_temp_wk_dir

In [5]:
#!aws s3 cp s3://aegovan-data/pubmed_asbtract/predictions_multi_95/pubmed19n0538.json.txt.json.prediction.json .

In [6]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def uploadfile(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [7]:
%%time

download_files(s3_prefix, local_temp_pred_dir)

CPU times: user 3min 53s, sys: 1min 52s, total: 5min 46s
Wall time: 4min 31s


In [8]:
!ls -l $local_temp_dir | wc -l

31


In [9]:
threshold_config = {
    "acetylation" : 0.83,
    "deubiquitination" :0.50,
    "methylation" :.85,
    "phosphorylation" : .98,
    "demethylation" :0.0,
    "dephosphorylation" :.85,
    "ubiquitination":0.3
}

In [10]:
%%time 

full_df = None
total_counts = {}
for f in os.listdir(local_temp_pred_dir):
    df = pd.read_json(os.path.join(local_temp_pred_dir, f), orient="records", lines=True )
    
    count_dict = df.groupby(["predicted"])["predicted"].count().to_dict()
    min_dict = df.groupby(["predicted"])["predicted_confidence"].min().to_dict()
    
    for k in count_dict:
        if k not in total_counts:
            total_counts[k] = {}
        total_counts[k]["count"] = total_counts[k].get("count", 0) + count_dict[k]
        total_counts[k]["min"] = min(total_counts[k].get("min", 1.0) , min_dict[k])



    # Filter below threshold items
    high_quality_frames = []
    for k,t in threshold_config.items():
        high_quality_frames.append(df.query("predicted == '{}' and predicted_confidence > {}".format(k, t)))
        
    high_quality_df = pd.concat(high_quality_frames)
    
    
    
    if full_df is None:
        full_df = high_quality_df
    else:
        full_df = pd.concat([high_quality_df, full_df])
        
    
    


CPU times: user 5min 30s, sys: 1min 17s, total: 6min 47s
Wall time: 8min 52s


In [11]:
df_counts = pd.DataFrame(total_counts)

In [12]:
df_counts.head()

,acetylation,dephosphorylation,deubiquitination,methylation,other,phosphorylation,ubiquitination,demethylation
count,2835.000000,17319.000000,1145.000000,13143.00000,9.854011e+06,404850.000000,354.000000,1.000000
min,0.166537,0.168799,0.162342,0.17894,1.734827e-01,0.170796,0.181956,0.377643


In [13]:
full_df.groupby(["predicted"])["predicted"].count().to_dict()

{'acetylation': 16,
 'demethylation': 1,
 'dephosphorylation': 180,
 'deubiquitination': 92,
 'methylation': 1909,
 'phosphorylation': 1252,
 'ubiquitination': 36}

In [14]:
full_df.groupby(["predicted"])["predicted_confidence"].describe()

,count,mean,std,min,25%,50%,75%,max
predicted,,,,,,,,
acetylation,16.0,0.853635,0.017940,0.834194,0.841396,0.846593,0.860633,0.906370
demethylation,1.0,0.377643,NaN,0.377643,0.377643,0.377643,0.377643,0.377643
dephosphorylation,180.0,0.898379,0.031899,0.850141,0.871749,0.892653,0.921433,0.962255
deubiquitination,92.0,0.560631,0.044044,0.500080,0.522097,0.547255,0.592547,0.695426
methylation,1909.0,0.888543,0.023966,0.850032,0.868216,0.886418,0.907715,0.946525
phosphorylation,1252.0,0.984180,0.002718,0.980010,0.981914,0.983852,0.986243,0.991165
ubiquitination,36.0,0.315345,0.011990,0.300192,0.304990,0.313962,0.319787,0.347559


In [15]:
full_df.shape

(3486, 16)

In [16]:
download_file(s3_data, local_temp_wk_dir)



In [17]:
data_file = os.path.join(local_temp_wk_dir, s3_data.split("/")[-1])
data_training_full_df = pd.read_json(data_file)

In [18]:
data_training_full_df.shape

(3381, 7)

In [19]:
full_df.shape

(3486, 16)

In [20]:
data_training_full_df.head(n=2)

,interactionId,interactionType,isNegative,participants,pubmedId,pubmedTitle,pubmedabstract
0,1585513,phosphorylation,false,"[{'uniprotid': 'Q10728', 'alias': [['mypt1_rat...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...
1,1585516,phosphorylation,false,"[{'uniprotid': 'O43293-1', 'alias': [['o43293-...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...


In [21]:
full_df.head(n=2)

,abstract,acetylation,confidence_scores,demethylation,dephosphorylation,deubiquitination,methylation,normalised_abstract,other,participant1Id,participant2Id,phosphorylation,predicted,predicted_confidence,pubmedId,ubiquitination
395,Fragile X syndrome is one of the most frequent...,0.804310,"{'acetylation': -0.21777061220000002, 'demethy...",0.621809,0.245314,0.514101,33.977565,Fragile X syndrome is one of the most frequent...,1.754072,P00995,Q06787,0.446770,methylation,0.874934,8372665,0.470480
2783,The experiments reported here document that th...,0.286923,"{'acetylation': -1.2485398501, 'demethylation'...",0.181673,0.359273,0.251972,0.231727,The experiments reported here document that th...,4.234165,P06400,P15172,308.552267,phosphorylation,0.981568,8381715,0.248378


In [22]:
full_df[~full_df.pubmedId.isin(data_training_full_df.pubmedId)].shape

(3253, 16)

In [23]:
full_df["PubmedInTrainingData"] = full_df.pubmedId.isin(data_training_full_df.pubmedId)

In [24]:
full_df.query("not PubmedInTrainingData").groupby(["predicted"])["predicted_confidence"].describe()

,count,mean,std,min,25%,50%,75%,max
predicted,,,,,,,,
acetylation,14.0,0.848446,0.009895,0.834194,0.841258,0.845236,0.854201,0.869485
dephosphorylation,133.0,0.888944,0.023922,0.850141,0.870096,0.887317,0.902896,0.943967
deubiquitination,88.0,0.557122,0.040684,0.500080,0.520021,0.545226,0.589662,0.659376
methylation,1888.0,0.888159,0.023650,0.850032,0.868155,0.885885,0.907304,0.944177
phosphorylation,1099.0,0.983993,0.002639,0.980010,0.981810,0.983620,0.986003,0.991165
ubiquitination,31.0,0.314249,0.011414,0.300192,0.304916,0.313602,0.317595,0.347559


In [25]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [26]:
full_df.to_csv("predictions.csv", index=False, header=True)

In [27]:
full_df[["abstract", "normalised_abstract", "participant1Id","participant2Id", "pubmedId", "predicted" ,"predicted_confidence" ]].sample(n=20)

,abstract,normalised_abstract,participant1Id,participant2Id,pubmedId,predicted,predicted_confidence
7511,"Gene activity is regulated by transcriptional and epigenetic mechanisms. A paper in 2005 by Zhang et al.(1) showed that STAT3 binds to the DNA methyl transferase, DNMT1 and their data indicated that STAT3 may cause epigenetic gene silencing by targeting DNMT1 to the PTPN6 promoter. Now, a paper by Lee et al.(2) has fleshed out the mechanism. They provide evidence that acetylation of STAT3 regulates the binding of DNMT1, CpG DNA methylation and regulation of several genes, including that encoding the estrogen receptor (ESR1) in breast cancer cells.","Gene activity is regulated by transcriptional and epigenetic mechanisms. A paper in 2005 by Zhang et al.(1) showed that P40763 binds to the DNA methyl transferase, P26358 and their data indicated that P40763 may cause epigenetic gene silencing by targeting P26358 to the P29350 promoter. Now, a paper by Lee et al.(2) has fleshed out the mechanism. They provide evidence that acetylation of P40763 regulates the binding of P26358, CpG DNA methylation and regulation of several genes, including that encoding the P03372 (P03372) in breast cancer cells.",P26358,P40763,24058781,methylation,0.870079
13650,"The cells in the preimplantation mammalian embryo undergo several rounds of fast cell division. Whether the known DNA repair pathways are active during these early stages of development where cell division is of primary importance, has not been fully established. Because of the important role of phosphorylated H2A.X (gammaH2A.X) in the DNA damage response as well as its putative role in assembly of embryonic chromatin, we analysed its distribution in the preimplantation mouse embryo. We found that H2A.X is highly phosphorylated throughout preimplantation development in the absence of any induced DNA damage. Moreover, gammaH2A.X levels vary significantly throughout the cell cycle. Interestingly, after the 4-cell stage, we detected high levels of H2A.X phosphorylation in mitosis, where telomeres appeared focally enriched with gammaH2A.X. In contrast, 53BP1, which is known to be recruited to DNA damage sites, is undetectable at mitotic chromosomes at these stages and its localisation changes upon blastocyst formation from mainly nuclear to cytoplasmic. We also show that 53BP1 and gammaH2A.X rarely colocalise, suggesting that the high levels of phosphorylation of H2A.X in the embryo might not be directly linked to the DNA damage response in the embryo. Our data suggest that phosphorylation of H2A.X is an important event in the fast dividing cells of the early embryo in the absence of any induced DNA damage. We discuss the possible consequences of these findings on the genome-wide chromatin remodelling that ocurs in the preimplantation mammalian embryo.","The cells in the preimplantation mammalian embryo undergo several rounds of fast cell division. Whether the known DNA repair pathways are active during these early stages of development where cell division is of primary importance, has not been fully established. Because of the important role of phosphorylated P16104 (gammaH2A.X) in the DNA damage response as well as its putative role in assembly of embryonic chromatin, we analysed its distribution in the preimplantation mouse embryo. We found that P16104 is highly phosphorylated throughout preimplantation development in the absence of any induced DNA damage. Moreover, gammaH2A.X levels vary significantly throughout the cell cycle. Interestingly, after the 4-cell stage, we detected high levels of P16104 phosphorylation in mitosis, where telomeres appeared focally enriched with gammaH2A.X. In contrast, Q12888, which is known to be recruited to DNA damage sites, is undetectable at mitotic chromosomes at these stages and its localisation changes upon blastocyst formation from mainly nuclear to cytoplasmic. We also show that Q12888 and gammaH2A.X rarely colocalise, suggesting that the high lev